### Libraries

In [ ]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop

In [ ]:
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.micro_architectures import *
from utils import *


# DATA

In [ ]:
start_time = '20220801'
end_time = '20230801'
scrap_date = interval_time(start_time, end_time)

In [ ]:
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)

In [ ]:
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date)

### Solving missing values with interpolation

In [ ]:
start_time_ = '2022-08-01 00:00:00'
end_time_ = '2023-08-03 23:59:00' 

In [ ]:
freq = '1T'

full_time_index = pd.date_range(start=start_time_, end=end_time_, freq=freq)

l1_sample = l1_sample.reindex(full_time_index)

l1_sample = l1_sample.interpolate(method='linear')

In [ ]:
l2_sample = l2_sample.reindex(full_time_index)

l2_sample = l2_sample.interpolate(method='linear')

# L1 (raw) data

## Minute based

In [ ]:
l1_sample

## Hour based

In [ ]:
l1_sample_hour = l1_sample.resample('60min').mean()
l1_sample_hour

# L2 (cleaned) data

## Minute based

In [ ]:
l2_sample

## Hour based

In [ ]:
l2_sample_hour = l2_sample.resample('60min').mean()
l2_sample_hour

# Dst data

In [ ]:
dst

# Kp data

In [ ]:
kp

# Datasets
### Descriptions:
**hn_dl**: hour normal dataloader

**mn_dl**: minute normal dataloader

**hr_dl**: minute normal dataloader

**mr_dl**: minute normal dataloader



In [ ]:
device = get_default_device()

In [ ]:
sequence_length_hour = 10  #hour
sequence_length_minute = 600 #minute
pred_length = 6 #hours

In [ ]:
##Normal
hour_Normal_dataset = NormalTrainingDataset(l1_sample_hour, dst, kp, sequence_length_hour, pred_length, hour = True)
minute_Normal_dataset = NormalTrainingDataset(l1_sample, dst, kp, sequence_length_minute, pred_length, hour = False)
##Refined(new method)
hour_Refined_dataset = RefinedTrainingDataset(l1_sample_hour, l2_sample_hour, dst,kp,sequence_length_hour, pred_length, hour = True)
minute_Refined_dataset = RefinedTrainingDataset(l1_sample, l2_sample, dst,kp,sequence_length_minute, pred_length, hour = False)

In [ ]:
#Test:15% training: 85%

test_size = round(0.15*len(hour_Normal_dataset))

train_hn_ds, test_hn_ds = random_split(hour_Normal_dataset , [len(hour_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hn_dl = DataLoader(train_hn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hn_dl = DeviceDataLoader(train_hn_dl, device)
test_hn_dl = DataLoader(test_hn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hn_dl = DeviceDataLoader(test_hn_dl, device)

In [ ]:
#Test:15% training: 85%

test_size = round(0.15*len(minute_Normal_dataset))

train_mn_ds, test_mn_ds = random_split(minute_Normal_dataset , [len(minute_Normal_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mn_dl = DataLoader(train_mn_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mn_dl = DeviceDataLoader(train_mn_dl, device)
test_mn_dl = DataLoader(test_mn_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mn_dl = DeviceDataLoader(test_mn_dl, device)


In [ ]:
#Test:15% training: 85%

test_size = round(0.15*len(hour_Refined_dataset))

train_hr_ds, test_hr_ds = random_split(hour_Refined_dataset , [len(hour_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hr_dl = DataLoader(train_hr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hr_dl = DeviceDataLoader(train_hr_dl, device)
test_hr_dl = DataLoader(test_hr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_hr_dl = DeviceDataLoader(test_hr_dl, device)

In [ ]:
#Test:15% training: 85%

test_size = round(0.15*len(minute_Refined_dataset))

train_mr_ds, test_mr_ds = random_split(minute_Refined_dataset , [len(minute_Refined_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_mr_dl = DataLoader(train_mr_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_mr_dl = DeviceDataLoader(train_mr_dl, device)
test_mr_dl = DataLoader(test_mr_ds, batch_size*2, num_workers=4, pin_memory=True)
test_mr_dl = DeviceDataLoader(test_mr_dl, device)

# Models

## Refined models with attention

### 1d Convolutional encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, input_size, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, input_size, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, 1, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, 29, *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, 1, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, 29, *architecture)

In [ ]:
RefinedCNN_min = to_device(RefinedArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
RefinedCNN_hour = to_device(RefinedArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedCNN_min_history = []

In [ ]:
RefinedCNN_min_history += RefinedCNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedCNN_hour_history = []

In [ ]:
RefinedCNN_hour_history += RefinedCNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep LSTM encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
##Bidirectional minute based models
deep_lstm_encoder_min_forward = DeepLSTM(hidden_size, input_size, batch_size, architecture)
deep_lstm_encoder_min_backward = DeepLSTM(hidden_size, input_size, batch_size, architecture)
bidirectional_deeplstm_encoder_min = BidirectionalRNNWithAttention(deep_lstm_encoder_min_forward, deep_lstm_encoder_min_backward)
##Bidirectional hour based models
deep_lstm_encoder_hour_forward = DeepLSTM(hidden_size, input_size, batch_size, architecture)
deep_lstm_encoder_hour_backward = DeepLSTM(hidden_size, input_size, batch_size, architecture)
bidirectional_deeplstm_encoder_hour = BidirectionalRNNWithAttention(deep_lstm_encoder_hour_forward, deep_lstm_encoder_hour_backward)
#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture) #the multiplying factor because concatenating hidden_states on bidirectional arch
deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

In [ ]:
RefinedLSTM_min = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
RefinedLSTM_hour = to_device(RefinedArchitecture(bidirectional_deeplstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedLSTM_min_history = []

In [ ]:
RefinedLSTM_min_history += RefinedLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedLSTM_hour_history = []

In [ ]:
RefinedLSTM_hour_history += RefinedLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
##Bidirectional minute based models
deep_gru_encoder_min_forward= DeepGRU(hidden_size, input_size, batch_size, architecture)
deep_gru_encoder_min_backward= DeepGRU(hidden_size, input_size, batch_size, architecture)
bidirectional_deepgru_encoder_min = BidirectionalRNNWithAttention(deep_gru_encoder_min_forward, deep_gru_encoder_min_backward)
##Bidirectional hour based models
deep_gru_encoder_hour_forward = DeepGRU(hidden_size, input_size, batch_size, architecture)
deep_gru_encoder_hour_backward = DeepGRU(hidden_size, input_size, batch_size, architecture)
bidirectional_deepgru_encoder_hour = BidirectionalRNNWithAttention(deep_gru_encoder_hour_forward, deep_gru_encoder_hour_backward)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

In [ ]:
RefinedGRU_min = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
RefinedGRU_hour = to_device(RefinedArchitecture(bidirectional_deepgru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedGRU_min_history = []

In [ ]:
RefinedGRU_min_history += RefinedGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedGRU_hour_history = []

In [ ]:
RefinedGRU_hour_history += RefinedGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
##Bidirectional minute encoders
deep_rnn_encoder_min_forward = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
deep_rnn_encoder_min_backward = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
bidirectional_deeprnn_encoder_min = BidirectionalRNNWithAttention(deep_rnn_encoder_min_forward,deep_rnn_encoder_min_backward)
##Bidirectional hour encoders
deep_rnn_encoder_hour_forward = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
deep_rnn_encoder_hour_backward = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
bidirectional_deeprnn_encoder_hour = BidirectionalRNNWithAttention(deep_rnn_encoder_hour_forward,deep_rnn_encoder_hour_backward)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

In [ ]:
RefinedVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
RefinedVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_deeprnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
RefinedVanillaRNN_min_history = []

In [ ]:
RefinedVanillaRNN_min_history += RefinedVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedVanillaRNN_hour_history = []

In [ ]:
RefinedVanillaRNN_hour_history += RefinedVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional non deep architectures

#### LSTM

In [ ]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

In [ ]:
RefinedNonDeepLSTM_min = to_device(RefinedArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
RefinedNonDeepLSTM_hour = to_device(RefinedArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
RefinedNonDeepLSTM_min_history = []

In [ ]:
RefinedNonDeepLSTM_min_history += RefinedNonDeepLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepLSTM_hour_history = []

In [ ]:
RefinedNonDeepLSTM_hour_history += RefinedNonDeepLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

#### GRU

In [ ]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)


In [ ]:
RefinedNonDeepGRU_min = to_device(RefinedArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
RefinedNonDeepGRU_hour = to_device(RefinedArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
RefinedNonDeepGRU_min_history = []

In [ ]:
RefinedNonDeepGRU_min_history += RefinedNonDeepGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepGRU_hour_history = []

In [ ]:
RefinedNonDeepGRU_hour_history += RefinedNonDeepGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

#### RNN

In [ ]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)


In [ ]:
RefinedNonDeepVanillaRNN_min = to_device(RefinedArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
RefinedNonDeepVanillaRNN_hour = to_device(RefinedArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
RefinedNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += RefinedNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
RefinedNonDeepVanillaRNN_hour_history = []

In [ ]:
RefinedNonDeepVanillaRNN_hour_history += RefinedNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

## Normal models with attention

### 1d Convolutional encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (16,8,8)

In [ ]:
#encoders
cnn_encoder_min = Simple1DCNN(architecture, input_size, hidden_size)
cnn_encoder_hour = Simple1DCNN(architecture, input_size, hidden_size)
#fc layer
cnn_fc_dst_min = DeepNeuralNetwork(hidden_size, 1, *architecture)
cnn_fc_kp_min = DeepNeuralNetwork(hidden_size, 29, *architecture)

cnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, 1, *architecture)
cnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, 29, *architecture)

In [ ]:
NormalCNN_min = to_device(NormalArchitecture(cnn_encoder_min, cnn_fc_dst_min, cnn_fc_kp_min), device)
NormalCNN_hour = to_device(NormalArchitecture(cnn_encoder_hour, cnn_fc_dst_hour, cnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalCNN_min_history = []

In [ ]:
NormalCNN_min_history += NormalCNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalCNN_hour_history = []

In [ ]:
NormalCNN_hour_history += NormalCNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep LSTM encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
deep_lstm_encoder_min = DeepLSTM(hidden_size, input_size, batch_size, architecture)
deep_lstm_encoder_hour = DeepLSTM(hidden_size, input_size, batch_size, architecture)
#fc layer
deep_lstm_fc_dst_min = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_lstm_fc_kp_min = DeepNeuralNetwork(hidden_size, 29, *architecture)

deep_lstm_fc_dst_hour = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_lstm_fc_kp_hour = DeepNeuralNetwork(hidden_size, 29, *architecture)

In [ ]:
NormalLSTM_min = to_device(NormalArchitecture(deep_lstm_encoder_min, deep_lstm_fc_dst_min, deep_lstm_fc_kp_min), device)
NormalLSTM_hour = to_device(NormalArchitecture(deep_lstm_encoder_hour, deep_lstm_fc_dst_hour, deep_lstm_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalLSTM_min_history = []

In [ ]:
NormalLSTM_min_history += NormalLSTM_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalLSTM_hour_history = []

In [ ]:
NormalLSTM_hour_history += NormalLSTM_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep GRU encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
deep_gru_encoder_min = DeepGRU(hidden_size, input_size, batch_size, architecture)
deep_gru_encoder_hour = DeepGRU(hidden_size, input_size, batch_size, architecture)
#fc layer
deep_gru_fc_dst_min = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_gru_fc_kp_min = DeepNeuralNetwork(hidden_size, 29, *architecture)

deep_gru_fc_dst_hour = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_gru_fc_kp_hour = DeepNeuralNetwork(hidden_size, 29, *architecture)

In [ ]:
NormalGRU_min = to_device(NormalArchitecture(deep_gru_encoder_min, deep_gru_fc_dst_min, deep_gru_fc_kp_min), device)
NormalGRU_hour = to_device(NormalArchitecture(deep_gru_encoder_hour, deep_gru_fc_dst_hour, deep_gru_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalGRU_min_history = []

In [ ]:
NormalGRU_min_history += NormalGRU_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalGRU_hour_history = []

In [ ]:
NormalGRU_hour_history += NormalGRU_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

### Bidirectional Deep Vanilla RNN encoder

In [ ]:
## Architecture
hidden_size = 10
input_size = 20
##nn architecture
architecture = (10,10,10)

In [ ]:
#encoders
deep_rnn_encoder_min = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
deep_rnn_encoder_hour = DeepVanillaRNN(hidden_size, input_size, batch_size, architecture)
#fc layer
deep_rnn_fc_dst_min = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_rnn_fc_kp_min = DeepNeuralNetwork(hidden_size, 29, *architecture)

deep_rnn_fc_dst_hour = DeepNeuralNetwork(hidden_size, 1, *architecture)
deep_rnn_fc_kp_hour = DeepNeuralNetwork(hidden_size, 29, *architecture)

In [ ]:
NormalVanillaRNN_min = to_device(RefinedArchitecture(deep_rnn_encoder_min, deep_rnn_fc_dst_min, deep_rnn_fc_kp_min), device)
NormalVanillaRNN_hour = to_device(RefinedArchitecture(deep_rnn_encoder_hour, deep_rnn_fc_dst_hour, deep_rnn_fc_kp_hour), device)

In [ ]:
##hyperparameters
epochs = 100
max_lr = 1e-3
weigth_decay = 1e-4
grad_clip = 1e-2
#opt_func = Adam
opt_func = RMSprop

In [ ]:
NormalVanillaRNN_min_history = []

In [ ]:
NormalVanillaRNN_min_history += NormalVanillaRNN_min.fit(epochs, max_lr, train_mn_dl, test_mn_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalVanillaRNN_hour_history = []

In [ ]:
NormalVanillaRNN_hour_history += NormalVanillaRNN_hour.fit(epochs, max_lr, train_hn_dl, test_hn_dl, weigth_decay, grad_clip, opt_func)

### Non deep bidirectional architectures with attention

#### LSTM

In [ ]:
non_deep_lstm_forward_min = LSTM(input_size, hidden_size)
non_deep_lstm_backward_min = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_min = BidirectionalRNNWithAttention(non_deep_lstm_forward_min, non_deep_lstm_backward_min)

non_deep_lstm_forward_hour = LSTM(input_size, hidden_size)
non_deep_lstm_backward_hour = LSTM(input_size, hidden_size)
bidirectional_lstm_encoder_hour = BidirectionalRNNWithAttention(non_deep_lstm_forward_hour, non_deep_lstm_backward_hour)
#fc layer
non_deep_lstm_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_lstm_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_lstm_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_lstm_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

In [ ]:
NormalNonDeepLSTM_min = to_device(NormalArchitecture(bidirectional_lstm_encoder_min, non_deep_lstm_fc_dst_min, non_deep_lstm_fc_kp_min), device)
NormalNonDeepLSTM_hour = to_device(NormalArchitecture(bidirectional_lstm_encoder_hour, non_deep_lstm_fc_dst_hour, non_deep_lstm_fc_kp_hour), device)

In [ ]:
NormalNonDeepLSTM_min_history = []

In [ ]:
NormalNonDeepLSTM_min_history += NormalNonDeepLSTM_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepLSTM_hour_history = []

In [ ]:
NormalNonDeepLSTM_hour_history += NormalNonDeepLSTM_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

#### GRU

In [ ]:
non_deep_gru_forward_min = GRU(input_size, hidden_size)
non_deep_gru_backward_min = GRU(input_size, hidden_size)
bidirectional_gru_encoder_min = BidirectionalRNNWithAttention(non_deep_gru_forward_min, non_deep_gru_backward_min)

non_deep_gru_forward_hour = GRU(input_size, hidden_size)
non_deep_gru_backward_hour = GRU(input_size, hidden_size)
bidirectional_gru_encoder_hour = BidirectionalRNNWithAttention(non_deep_gru_forward_hour, non_deep_gru_backward_hour)
#fc layer
non_deep_gru_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_gru_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_gru_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_gru_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)


In [ ]:
NormalNonDeepGRU_min = to_device(NormalArchitecture(bidirectional_gru_encoder_min, non_deep_gru_fc_dst_min, non_deep_gru_fc_kp_min), device)
NormalNonDeepGRU_hour = to_device(NormalArchitecture(bidirectional_gru_encoder_hour, non_deep_gru_fc_dst_hour, non_deep_gru_fc_kp_hour), device)

In [ ]:
NormalNonDeepGRU_min_history = []

In [ ]:
NormalNonDeepGRU_min_history += NormalNonDeepGRU_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepGRU_hour_history = []

In [ ]:
NormalNonDeepGRU_hour_history += NormalNonDeepGRU_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

#### RNN

In [ ]:
non_deep_rnn_forward_min = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_min = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_min = BidirectionalRNNWithAttention(non_deep_rnn_forward_min, non_deep_rnn_backward_min)

non_deep_rnn_forward_hour = VanillaRNN(input_size, hidden_size)
non_deep_rnn_backward_hour = VanillaRNN(input_size, hidden_size)
bidirectional_rnn_encoder_hour = BidirectionalRNNWithAttention(non_deep_rnn_forward_hour, non_deep_rnn_backward_hour)
#fc layer
non_deep_rnn_fc_dst_min = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_rnn_fc_kp_min = DeepNeuralNetwork(2*hidden_size, 29, *architecture)

non_deep_rnn_fc_dst_hour = DeepNeuralNetwork(2*hidden_size, 1, *architecture)
non_deep_rnn_fc_kp_hour = DeepNeuralNetwork(2*hidden_size, 29, *architecture)


In [ ]:
NormalNonDeepVanillaRNN_min = to_device(NormalArchitecture(bidirectional_rnn_encoder_min, non_deep_rnn_fc_dst_min, non_deep_rnn_fc_kp_min), device)
NormalNonDeepVanillaRNN_hour = to_device(NormalArchitecture(bidirectional_rnn_encoder_hour, non_deep_rnn_fc_dst_hour, non_deep_rnn_fc_kp_hour), device)

In [ ]:
NormalNonDeepVanillaRNN_min_history = []

In [ ]:
RefinedNonDeepVanillaRNN_min_history += NormalNonDeepVanillaRNN_min.fit(epochs, max_lr, train_mr_dl, test_mr_dl, weigth_decay, grad_clip, opt_func)

In [ ]:
NormalNonDeepVanillaRNN_hour_history = []

In [ ]:
NormalNonDeepVanillaRNN_hour_history += NormalNonDeepVanillaRNN_hour.fit(epochs, max_lr, train_hr_dl, test_hr_dl, weigth_decay, grad_clip, opt_func)

# Conclusion